In [1]:
%load_ext autoreload
%autoreload 2
import mmcv
from mmdet3d.apis import inference_mono_3d_detector, init_model
from mmdet3d.registry import VISUALIZERS
from mmengine import load, dump
from mmdet3d.registry import VISUALIZERS
import cv2
import numpy as np
import sys
from sensus.tools.data_processor import DataProcessor
from sensus.tools.visualizer import ImageVisualizer, draw_monodetection_labels, draw_monodetection_results

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
file = '002238'
dataset = 'dair'
if dataset == 'kitti':
    img_file = '/home/javier/datasets/kitti/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/kitti/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/kitti/training/calib/'+ file + '.txt'
    pitch = 0.0
elif dataset == 'dair':
    img_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/calib/'+ file + '.txt'
    pitch = 0.2031

ann_file = '/home/javier/sensus-loci/sensus/notebooks/002238.pkl'
cam_type = 'CAM_BACK'

## Draw Labels

In [7]:
draw_monodetection_labels(img_file, calib_file, labels_file, num_cars=15, pitch=pitch, thickness=1)

[1.48235, 1.937682, 4.241219]
[1.712159, 2.069001, 4.347082]
[1.795317, 2.09444, 4.311423]
[1.403052, 2.113689, 4.268273]
[1.456858, 1.816689, 4.416424]
[3.19582, 3.196433, 8.162645]
[1.910056, 2.262334, 4.455209]
[1.645071, 2.11366, 4.337559]
[1.340647, 2.002429, 4.191494]
[3.713664, 3.73082, 7.499055]
[1.576959, 1.815553, 4.189867]
[1.421129, 2.02112, 4.740007]
[1.090961, 2.16069, 4.240069]


## Inference

In [3]:
config_file = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d.py'

checkpoint_file = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth'

device = 'cuda:0'
model_mmdet = init_model(config_file, checkpoint_file, device=device)
result = inference_mono_3d_detector(model_mmdet, img_file, ann_file, cam_type)

# init visualizer
visualizer_mmdet = VISUALIZERS.build(model_mmdet.cfg.visualizer)
visualizer_mmdet.dataset_meta = model_mmdet.dataset_meta

Loads checkpoint by local backend from path: /home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.projs.0.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.nodes.0.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.0.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.1.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.0.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.1.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_2.projs.0.conv is upgraded to version 2.
11/23 17:38:29 - mmengine - INFO - ModulatedDef

/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [6]:
print(result.pred_instances_3d)

<InstanceData(

    META INFORMATION

    DATA FIELDS
    scores_3d: tensor([0.5217, 0.4985, 0.4795, 0.4576, 0.4507, 0.4311, 0.4160, 0.4013, 0.3731,
                0.3315, 0.3309, 0.3294, 0.3123, 0.3022], device='cuda:0')
    bboxes_3d: CameraInstance3DBoxes(
            tensor([[-20.4564, -13.7553,  92.5255,   4.2626,   1.5427,   1.8652,   1.6037],
                [ -2.2950,  -5.7455,  57.8315,   4.2426,   1.5058,   1.8570,  -1.5340],
                [-17.2295,  -2.7285,  44.4448,   4.3859,   1.4860,   1.8846,   1.6451],
                [  8.0622, -11.7175,  84.0199,   6.3734,   2.6787,   2.5188,  -1.5328],
                [  0.9558,  -5.3186,  56.0010,   4.2391,   1.7275,   1.9132,  -1.5417],
                [-24.0466,  -5.9347,  58.6978,   4.4513,   1.7328,   1.9342,   1.5913],
                [  1.2116,  -0.2200,  34.2346,   4.2614,   1.8245,   1.9512,  -1.5552],
                [-20.3021,  -7.2970,  64.5863,   4.2710,   1.4942,   1.8839,   1.6138],
                [-17.0324, -16.

## MMDet Viz

In [9]:
img = mmcv.imread(img_file)
img = mmcv.imconvert(img, 'bgr', 'rgb')
data_input = dict(img=img)
# show the results
visualizer_mmdet.add_datasample(
    'result',
    data_input,
    data_sample=result,
    draw_gt=False,
    show=True,
    wait_time=0,
    out_file='result_mmdet',
    pred_score_thr= 0.30,
    vis_task='mono_det')

/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:812: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(


## Sensus Viz

In [10]:
draw_monodetection_results(img_file, calib_file, result, score=0.49, pitch=pitch, thickness=2)